In [109]:
from scipy.stats import chi2_contingency
import calendar
import prince
months = list(map(lambda x: x.lower(), calendar.month_name))[1:]
import pandas as pd
from pandas import DataFrame

import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

import matplotlib.pylab as pylab




params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 10),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
pylab.rcParams.update(params)

import os

project_folder = f'{os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))}/data/process'
import warnings

warnings.filterwarnings(action='ignore')

from IPython.core.display import display, HTML
# Set the display width to fit the entire notebook width
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import Markdown
from IPython.display import display_html


predictors ={'Q13':'value_approx','Q14':'is_electric','Q15':'bicycle_type','Q18':'is_recover',  'Q28':'seasons', 'Q29':'purpose', 'age_groups': 'age_groups', 'Q35': 'gender', 'Q36': 'income','Q37':'nm_bikes', 'Q38': 'education','country':'country'}
dependent_vars = {'Q25':'is_replaced','sustainable':'mode_alt', 'Q31':'post_act'}
all_vars = predictors.copy()
all_vars.update(dependent_vars)

In [118]:
############ RUN FOR THE FIRST TIME ONLY#################
# Change col names and leave only relevant cols as well as delete unnecessary spaces and parenthesis (except age group)
cols_names = list(all_vars.values())
data_init = pd.read_csv(f'{project_folder}/new_data/new_data_2.csv')
temp = data_init['age_groups']
d_analysis = data_init.rename(columns=all_vars)[cols_names].astype(str).replace(r"\(.*?\)", "").astype(str).replace(":", "").apply(lambda row: [d.split('(')[0].strip() if '(' in d else d.strip() for d in row])
d_analysis['age_groups'] = temp

# To enhance clarity and intuitive understanding, swap the labels between Class 0 and Class 1.
def swap(t):
    return 1 if t==0 else 0 if t==1 else t

data_init['ridership'] = data_init.Q28.apply(swap)

# This method translate the alternative data into how much the alternative is sustainable
def sus_alter(row):
    alt_stat = ['sustainable','semi','non sustainable']
    if row in  ['Walk','Cycle, personal bicycle','Cycle, rental bicycle','Cycle, public bike share','Micro mobility']:
        return alt_stat[0]
    elif row in ['Transit','Motorcycle or scooter']:
        return alt_stat[1]
    elif row in ['Car, as a driver','Car, as a passenger','Taxi / Ride-hailing service']:
        return alt_stat[2]
    else:
        return

data_init['sustainable'] = data_init['mode_alt'].apply(sus_alter)
data_init['sustainable'].value_counts()

d_analysis.to_csv(f'{project_folder}/new_data/new_data_3.csv')



In [125]:
data_init['mode_alt']


KeyError: 'mode_alt'

In [3]:
# read data
merge_q = pd.read_csv(f'{project_folder}/new_data/new_data_3.csv')

non sustainable            681
sustainable                620
Didn’t make those trips    221
semi                       177
Don’t know/not sure         29
Name: sustainable, dtype: int64

In [90]:
# organize and populate @more_data  dictionary
more_data ={item:[False,False] for  item in predictors.values()}

# Code relevant for reindex
# Bicycle attributes
q= 'value_approx'
df_t = DataFrame(merge_q[q].unique()).dropna().sort_values(by=0, ascending=0).reset_index(drop=True)
more_data[q][0] = pd.concat([df_t.iloc[0], df_t.iloc[6], df_t.iloc[4], df_t.iloc[8], df_t.iloc[7], df_t.iloc[5], df_t.iloc[1:4].sort_values(by=0, ascending=1)])[0].to_list()

# Demographic
q = 'education'
df_t = DataFrame(merge_q[q].unique()).dropna().sort_values(by=0, ascending=0).reset_index(drop=True)
more_data[q][0]= pd.concat([df_t.iloc[6], df_t.iloc[0:2], df_t.iloc[4], df_t.iloc[3:1:-1], df_t.iloc[5]])[0].to_list()
q = 'age_groups'
more_data[q][0] = DataFrame(merge_q[q].unique()).dropna()[::-1][0].to_list()
q= 'income'
df_t = DataFrame(merge_q[q].unique()).dropna().sort_values(by=0, ascending=0).reset_index(drop=True)
more_data[q][0]= pd.concat([df_t.iloc[0], df_t.iloc[6:2:-1], df_t.iloc[8:6:-1], df_t.iloc[9], df_t.iloc[1:3]])[0].to_list()
q = 'nm_bikes'
df_t = DataFrame(merge_q[q].unique()).dropna().sort_values(by=0, ascending=0).reset_index(drop=True)
more_data[q][0]= pd.concat([df_t.iloc[0], df_t.iloc[3], df_t.iloc[1:3], df_t.iloc[5:], df_t.iloc[4]])[0].to_list()
# In case where only several cols are relevant
more_data['bicycle_type'][1]=['Hybrid/City/Dutch','Mountain','Road','Gravel/cyclocross']

{'value_approx': [False, False],
 'is_electric': [False, False],
 'bicycle_type': [False, False],
 'is_recover': [False, False],
 'seasons': [False, False],
 'purpose': [False, False],
 'age_groups': [False, False],
 'gender': [False, False],
 'income': [False, False],
 'nm_bikes': [False, False],
 'education': [False, False],
 'country': [False, False]}

In [112]:

class MyData:
    """
   The class organizes and explores the data, allowing for the creation of cross-tabulations.
    """
    def __init__(self, var_0, data, com_data):
        """

        :param var_0: The main variable that should not be replaced frequently throughout the program.
        :param data: Data frame
        :param com_data: provides more information when necessary to create more adaptable analysis
        """
        self.cols_name = [var_0,'']
        self.merge_q= data
        self.more_data = com_data

    def explore_data(self,cross_tab= True):
        r"""
        Clean the data and print cross_tab if it is required
        :param cross_tab:
        :return:
        """
        # Based on these variables, the analysis can be employed.
        cross_cols_nm =self.cols_name
        cls_to_use = self.more_data[cross_cols_nm[1]][1]
        reindex_temp = self.more_data[cross_cols_nm[1]][0]
        only_rel_f= self.merge_q[cross_cols_nm].dropna()

        if cross_tab:
            if cls_to_use:
                only_rel_f = only_rel_f[only_rel_f[cross_cols_nm[1]].isin(cls_to_use)]
            if reindex_temp:
                # For ordinal categories, reindexing the columns is essential to align them in the required order.
                return only_rel_f[cross_cols_nm[1]].value_counts().reindex(reindex_temp), (pd.crosstab(only_rel_f[cross_cols_nm[0]], only_rel_f[cross_cols_nm[1]], normalize='columns') * 100).astype(int).reindex(columns=reindex_temp)
            else:
                return only_rel_f[cross_cols_nm[1]].value_counts(),(pd.crosstab(only_rel_f[cross_cols_nm[0]], only_rel_f[cross_cols_nm[1]], normalize='columns') * 100).astype(int)

        return only_rel_f

In [64]:
merge_q.columns

Index(['Unnamed: 0', 'value_approx', 'is_electric', 'bicycle_type',
       'is_recover', 'seasons', 'purpose', 'age_groups', 'gender', 'income',
       'nm_bikes', 'education', 'country', 'is_replaced', 'mode_alt',
       'post_act', 'sustainable'],
      dtype='object')

In [96]:
dependent_vars.key

dict_items([('Q25', 'is_replaced'), ('Q30', 'mode_alt'), ('Q31', 'post_act')])

In [116]:

dependent_names = list(dependent_vars.values())
predictors_names = list(predictors.values())
my_data = MyData(dependent_names[0],merge_q,more_data)
for dep in dependent_names:
    print(f'\n{dep}\n')
    writer = pd.ExcelWriter(f'output_2/{dep}.xlsx')
    my_data.cols_name[0] = dep
    for pre in predictors_names:
        print(pre)
        my_data.cols_name[1] = pre
        value_counts,cross_tab = my_data.explore_data()
        value_counts.to_excel(writer, sheet_name=pre)
        cross_tab.to_excel(writer, sheet_name=pre, startrow=value_counts.shape[0] + 2)
    writer.save()





is_replaced

value_approx
is_electric
bicycle_type
is_recover
seasons
purpose
age_groups
gender
income
nm_bikes
education
country

mode_alt

value_approx
is_electric
bicycle_type
is_recover
seasons
purpose
age_groups
gender
income
nm_bikes
education
country

post_act

value_approx
is_electric
bicycle_type
is_recover
seasons
purpose
age_groups
gender
income
nm_bikes
education
country


In [ ]:
# Parameters
cols_name = [q_0,q_1]
cols_to_work_with = ['Hybrid/City/Dutch','Mountain','Road','Gravel/cyclocross']
only_rel,res =explore_data(is_select_rows=['Hybrid/City/Dutch','Mountain','Road','Gravel/cyclocross'])
sus.cols_name[1] = 'is_electric'
only_rel,res =sus.explore_data()
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>There is an inverse relationship between the degree of electrification of the bike and the likelihood of the owner choosing a sustainable alternative. The higher the level of electrification, the lower the probability of opting for a sustainable alternative, and vice versa</li></ul>'
display(Markdown(text))

In [84]:
# Parameters
sus.cols_name[1] = 'is_electric'
only_rel,res =sus.explore_data()
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>There is an inverse relationship between the degree of electrification of the bike and the likelihood of the owner choosing a sustainable alternative. The higher the level of electrification, the lower the probability of opting for a sustainable alternative, and vice versa</li></ul>'
display(Markdown(text))

['sustainable', 'is_electric']
​No                                    1482
Yes, with pedal-assist and throttle     115
Yes, with pedal-assist                   90
Name: is_electric, dtype: int64


is_electric,"Yes, with pedal-assist","Yes, with pedal-assist and throttle",​No
sustainable,,,
Didn’t make those trips,5,7,13
Don’t know/not sure,2,0,1
non sustainable,46,56,37
semi,12,9,9
sustainable,33,25,36


<h1 style="font-size:22px">Arguments</h1><ul> <li>There is an inverse relationship between the degree of electrification of the bike and the likelihood of the owner choosing a sustainable alternative. The higher the level of electrification, the lower the probability of opting for a sustainable alternative, and vice versa</li></ul>

In [75]:
sus.cols_name

['sustainable', 'is_electric']

In [ ]:

# Parameters
q_0 = 'sustainable'
q_1 = 'value_approx'

cols_name = [q_0,q_1]
only_rel,res =explore_data(is_reindex= True)
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>Owners with bikes valued at less than $250 do not have the luxury of considering non-sustainable alternatives</li> <li> With the exception of bikes valued at more than 7000 bucks there is a negative correlation between the value of the bicycle and the likelihood of the owner opting for a sustainable alternative</li> </ul>'
display(Markdown(text))

## Variable - Is Replaced

In [35]:
# Parameters
q_0 = 'is_replaced'
q_1 = 'purpose'

cols_name = [q_0,q_1]
only_rel,res =explore_data()
res


Recreation or exercise                                         720
Transportation                                                 540
Equal mix of recreation, exercise, and transportation trips    454
Don’t know/not sure                                             10
Name: purpose, dtype: int64


purpose,Don’t know/not sure,"Equal mix of recreation, exercise, and transportation trips",Recreation or exercise,Transportation
is_replaced,,,,
No,80,31,31,27
"Yes, but the replacement was something cheaper",10,25,18,29
"​Yes, and I replaced it with exactly what was stolen or something more expensive",10,42,49,43


In [34]:
# Parameters
q_0 = 'is_replaced'
q_1 = 'seasons'


cols_name = [q_0,q_1]
only_rel,res =explore_data()
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>People with low ridership frequency has considerably less intended to replace their bikes </li></ul>'
display(Markdown(text))

0.0    796
2.0    663
1.0    226
Name: seasons, dtype: int64


seasons,0.0,1.0,2.0
is_replaced,,,
No,23,65,27
"Yes, but the replacement was something cheaper",24,13,25
"​Yes, and I replaced it with exactly what was stolen or something more expensive",52,21,46


<h1 style="font-size:22px">Arguments</h1><ul> <li>People with low ridership frequency has considerably less intended to replace their bikes </li></ul>

In [33]:
# Parameters
q_0 = 'is_replaced'
q_1 = 'country'

cols_name = [q_0,q_1]
only_rel,res =explore_data()
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>Maybe to add city level analysis</li></ul>'
display(Markdown(text))

USA    1476
CAN     336
Name: country, dtype: int64


country,CAN,USA
is_replaced,,
No,33,30
"Yes, but the replacement was something cheaper",23,23
"​Yes, and I replaced it with exactly what was stolen or something more expensive",43,46


<h1 style="font-size:22px">Arguments</h1><ul> <li>Maybe to add city level analysis</li></ul>

In [179]:
# Parameters
q_0 = 'is_replaced'
q_1 = 'nm_bikes'

cols_name = [q_0,q_1]
only_rel,res =explore_data(is_reindex=True)
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>individuals with zero bikes expectedly did not replace their bikes</li> <li> People with more bikes also tend to replace their bikes with the same or higher-value bicycle models and vice versa</li></ul>'
display(Markdown(text))

Zero                      211
One                       642
Two                       404
Three                     199
Four                      108
Five or more              139
I prefer to not answer     11
Name: nm_bikes, dtype: int64


nm_bikes,Zero,One,Two,Three,Four,Five or more,I prefer to not answer
is_replaced,,,,,,,
No,92,22,20,23,20,20,45
"Yes, but the replacement was something cheaper",3,29,26,26,20,19,0
"​Yes, and I replaced it with exactly what was stolen or something more expensive",3,48,52,50,59,60,54


<h1 style="font-size:22px">Arguments</h1><ul> <li>individuals with zero bikes expectedly did not replace their bikes</li> <li> People with more bikes also tend to replace their bikes with the same or higher-value bicycle models and vice versa</li></ul>

In [180]:
# Parameters
q_0 = 'is_replaced'
q_1 = 'income'


cols_name = [q_0,q_1]
only_rel,res =explore_data(is_reindex=True)
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>individuals with high income tend more to replace their bikes </li> <li> People with low income (less then $35000) are usually replace their bikes with cheaper model</li><li>The higher the income is the higher the likelihood to replaced the bike in the same of more expensive bike</li></ul>'
display(Markdown(text))

Under $20,000 per year                     83
Between $20,000 and $34,999 per year      110
Between $35,000 and $49,999 per year       92
Between $50,000 and $74,999 per year      218
Between $75,000 and $99,999 per year      231
Between $100,000 and $149,999 per year    268
Between $150,000 and $199,999 per year    174
$200,000 or more per year                 357
I prefer to not answer                    158
Don’t know/not sure                        17
Name: income, dtype: int64


income,"Under $20,000 per year","Between $20,000 and $34,999 per year","Between $35,000 and $49,999 per year","Between $50,000 and $74,999 per year","Between $75,000 and $99,999 per year","Between $100,000 and $149,999 per year","Between $150,000 and $199,999 per year","$200,000 or more per year",I prefer to not answer,Don’t know/not sure
is_replaced,,,,,,,,,,
No,34,36,36,33,32,32,27,22,31,41
"Yes, but the replacement was something cheaper",44,35,23,27,20,20,21,19,19,35
"​Yes, and I replaced it with exactly what was stolen or something more expensive",20,28,39,38,47,47,51,57,49,23


<h1 style="font-size:22px">Arguments</h1><ul> <li>individuals with high income tend more to replace their bikes </li> <li> People with low income (less then $35000) are usually replace their bikes with cheaper model</li><li>The higher the income is the higher the likelihood to replaced the bike in the same of more expensive bike</li></ul>

In [148]:
# Parameters
q_0 = 'is_replaced'
q_1 = 'age_groups'



cols_name = [q_0,q_1]
only_rel,res =explore_data(is_reindex=True)
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>There is clear correlation between the education to likelihood of replacing with the same or more expensive bike </li></ul>'
display(Markdown(text))

Children (<13)            5
Adolescents (13-17)      18
Young adults (18-24)     94
Adults (25-34)          496
Adults (35-44)          449
Adults (45-54)          310
Adults (55-64)          178
Older adults (>64)      262
Name: age_groups, dtype: int64


age_groups,Children (<13),Adolescents (13-17),Young adults (18-24),Adults (25-34),Adults (35-44),Adults (45-54),Adults (55-64),Older adults (>64)
is_replaced,,,,,,,,
No,0,22,38,34,31,25,25,31
"Yes, but the replacement was something cheaper",0,44,39,24,19,27,22,18
"​Yes, and I replaced it with exactly what was stolen or something more expensive",100,33,22,41,48,47,52,50


<h1 style="font-size:22px">Arguments</h1><ul> <li>There is clear correlation between the education to likelihood of replacing with the same or more expensive bike </li></ul>

In [138]:
# Parameters
q_0 = 'is_replaced'
q_1 = 'education'


cols_name = [q_0,q_1]
only_rel,res =explore_data(is_reindex=True)
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>There is clear correlation between the education to likelihood of replacing with the same or more expensive bike </li></ul>'
display(Markdown(text))

Some high school or less                    32
Graduated high school                       53
Some university                            121
Associate’s/vocational/technical degree    110
Bachelor’s degree                          662
Graduate degree                            709
I prefer to not answer                      25
Name: education, dtype: int64


education,Some high school or less,Graduated high school,Some university,Associate’s/vocational/technical degree,Bachelor’s degree,Graduate degree,I prefer to not answer
is_replaced,,,,,,,
No,31,39,40,37,33,24,24
"Yes, but the replacement was something cheaper",46,26,24,19,23,23,12
"​Yes, and I replaced it with exactly what was stolen or something more expensive",21,33,34,43,43,51,64


<h1 style="font-size:22px">Arguments</h1><ul> <li>I can use this data to show where there is no differences</li></ul>

In [104]:
# Parameters
q_0 = 'is_replaced'
q_1 = 'gender'

cols_name = [q_0,q_1]
only_rel,res =explore_data()
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>I can use this data to show where there is no differences</li></ul>'
display(Markdown(text))

Man                          1046
Woman                         609
Prefer not to say              35
Non-binary / third gender      21
Name: gender, dtype: int64


gender,Man,Non-binary / third gender,Prefer not to say,Woman
is_replaced,,,,
No,29,23,45,31
"Yes, but the replacement was something cheaper",22,42,28,25
"​Yes, and I replaced it with exactly what was stolen or something more expensive",47,33,25,43


<h1 style="font-size:22px">Arguments</h1><ul> <li>I can use this data to show where there is no differences</li></ul>

In [102]:
# Parameters
q_0 = 'is_replaced'
q_1 = 'is_recover'

cols_name = [q_0,q_1]
only_rel,res =explore_data()
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>33% of those who recovered their bike still replace their bikes and usually to the same of higher- value bikes </li></ul>'
display(Markdown(text))

No     1535
Yes     276
Name: is_recover, dtype: int64


is_recover,No,Yes
is_replaced,,
No,25,57
"Yes, but the replacement was something cheaper",25,10
"​Yes, and I replaced it with exactly what was stolen or something more expensive",48,32


<h1 style="font-size:22px">Arguments</h1><ul> <li>33% of those who recovered their bike still replace their bikes and usally to the same of higher- value bikes </li></ul>

In [100]:
# Parameters
q_0 = 'is_replaced'
q_1 = 'bicycle_type'

cols_name = [q_0,q_1]
cols_to_work_with = ['Hybrid/City/Dutch','Mountain','Road','Gravel/cyclocross']
only_rel,res =explore_data(is_select_rows=True)
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>Could be compare to the price</li></ul>'
display(Markdown(text))

Hybrid/City/Dutch    543
Mountain             540
Road                 373
Gravel/cyclocross    108
Name: bicycle_type, dtype: int64


bicycle_type,Gravel/cyclocross,Hybrid/City/Dutch,Mountain,Road
is_replaced,,,,
No,24,30,31,27
"Yes, but the replacement was something cheaper",20,26,22,23
"​Yes, and I replaced it with exactly what was stolen or something more expensive",55,43,46,49


<h1 style="font-size:22px">Arguments</h1><ul> <li>Could be compare to the price</li></ul>

In [97]:
merge_q[merge_q[cols_name[1]].isin(cols_to_work_with)]

,Unnamed: 0,value_approx,is_electric,bicycle_type,is_recover,seasons,purpose,age_groups,gender,income,nm_bikes,education,country,is_replaced,mode_alt,post_act
0,0,$1000-$1999,"Yes, with pedal-assist and throttle",Road,No,2.0,Recreation or exercise,Older adults,Woman,"Between $20,000 and $34,999 per year",One,Graduated high school,CAN,"​Yes, and I replaced it with exactly what was ...",Didn’t make those trips,More often
2,2,$2000-3999,​No,Road,No,2.0,Recreation or exercise,Older adults,Man,"Between $100,000 and $149,999 per year",Two,Graduate degree,USA,"​Yes, and I replaced it with exactly what was ...","Car, as a driver",About the same / no change
3,3,$500-$999,​No,Hybrid/City/Dutch,No,2.0,Recreation or exercise,Older adults,Woman,I prefer to not answer,One,Graduate degree,CAN,"​Yes, and I replaced it with exactly what was ...","Cycle, personal bicycle",About the same / no change
4,4,$1000-$1999,​No,Mountain,No,0.0,"Equal mix of recreation, exercise, and transpo...",Older adults,Man,"Between $100,000 and $149,999 per year",Two,Graduate degree,USA,"​Yes, and I replaced it with exactly what was ...","Cycle, personal bicycle",About the same / no change
5,5,$1000-$1999,​No,Road,No,0.0,Recreation or exercise,Older adults,Man,"Between $50,000 and $74,999 per year",Two,Graduate degree,USA,"Yes, but the replacement was something cheaper","Car, as a driver",About the same / no change
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1815,1815,$1000-$1999,​No,Hybrid/City/Dutch,No,NaN,NaN,Older adults,NaN,NaN,NaN,NaN,USA,"Yes, but the replacement was something cheaper",NaN,NaN
1816,1816,$2000-3999,"Yes, with pedal-assist and throttle",Hybrid/City/Dutch,No,0.0,"Equal mix of recreation, exercise, and transpo...",Older adults,NaN,NaN,NaN,NaN,USA,"​Yes, and I replaced it with exactly what was ...","Car, as a driver",About the same / no change
1818,1818,$500-$999,​No,Mountain,No,2.0,"Equal mix of recreation, exercise, and transpo...",Older adults,Woman,"Under $20,000 per year",Two,Graduate degree,USA,"Yes, but the replacement was something cheaper","Car, as a driver",Less often
1819,1819,$2000-3999,​No,Mountain,No,NaN,NaN,Older adults,NaN,NaN,NaN,NaN,CAN,No,NaN,NaN


In [92]:
# Parameters
q_0 = 'is_replaced'
q_1 = 'is_electric'

cols_name = [q_0,q_1]
only_rel,res =explore_data(True)
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>Unclear results</li></ul>'
display(Markdown(text))



​No                                    1553
Yes, with pedal-assist and throttle     120
Yes, with pedal-assist                   96
Name: is_electric, dtype: int64


is_electric,"Yes, with pedal-assist","Yes, with pedal-assist and throttle",​No
is_replaced,,,
No,25,32,30
"Yes, but the replacement was something cheaper",13,14,24
"​Yes, and I replaced it with exactly what was stolen or something more expensive",61,53,44


<h1 style="font-size:22px">Arguments</h1><ul> <li>Unclear results</li></ul>

In [87]:

# Parameters
q_0 = 'is_replaced'
q_1 = 'value_approx'

cols_name = [q_0,q_1]


only_rel,res =explore_data(is_reindex= True)
res
text = '<h1 style="font-size:22px">Arguments</h1><ul> <li>Regardless to bike value, owners tend to replace their bikes and generally with the same or more expensive bike </li> <li>This behavior is even more dominant in owners with high-value bikes</li></ul>'
display(Markdown(text))


Less than $250             77
$250-$499                 244
$500-$999                 520
$1000-$1999               426
$2000-3999                296
$4000-6999                146
$7000 or more              96
Don’t know/not sure         5
I prefer to not answer      2
Name: value_approx, dtype: int64


value_approx,Less than $250,$250-$499,$500-$999,$1000-$1999,$2000-3999,$4000-6999,$7000 or more,Don’t know/not sure,I prefer to not answer
is_replaced,,,,,,,,,
No,32,36,31,30,30,24,21,40,0
"Yes, but the replacement was something cheaper",25,27,26,23,18,16,18,20,50
"​Yes, and I replaced it with exactly what was stolen or something more expensive",41,36,41,45,51,58,59,40,50


<h1 style="font-size:22px">Arguments</h1><ul> <li>Regardless to bike value, owners tend to replace their bikes and generally with the same or more expensive bike </li> <li>This behavior is even more dominant in owners with high-value bikes</li></ul>

In [77]:
#
def calculate_chi_cross_tab(two_var_tables):
    r"""
    This function creates new groups based on the data in @vars, cross tab them and calculate Chi-square
    :return:
    """

    # the new names for the variables which be presented on the cross tabulation tabel
    new_names = [t['var'] for t in my_vars]
    two_var_tables[new_names] = two_var_tables.apply(lambda x: [x[t['q']] if x[t['q']] in t['group'] else t['group2'] for t in my_vars],axis=1,result_type='expand')
    cont_table = pd.crosstab([two_var_tables[t['var']] for t in my_vars[:-1]],two_var_tables[my_vars[-1]['var']],normalize='columns')*100

    # perform chi-square test of independence
    chi2, p, dof, expected = chi2_contingency(cont_table)

    # print the test results
    print(f"Chi-square statistic: {chi2:.2f}")
    print(f"P-value: {p:.5f}")
    print(cont_table)
    if to_print:
        cont_table.to_csv(to_print)

def prep_for_chi(only_rel_f,grp:dict):
    r"""
    this function store and set the data so only the relevant data will be used to calculate chi -square
    :param only_rel_f:
    :return:
    """
    # working only with information stored in list in @rel_data for a specific col
    for i in range(1,len(rel_data),2):
        if rel_data[i]:
            only_rel_f= only_rel_f[only_rel_f[rel_data[i-1][0]].isin(rel_data[i])] if rel_data[i-1][1] else only_rel_f[~only_rel_f[rel_data[i-1][0]].isin(rel_data[i])]
    # group data
    for q_temp in grp.keys():
        grp_inf = grp[q_temp]
        only_rel_f[q_temp] = only_rel_f[q_temp].apply(lambda x:grp_inf[0] if x in grp_inf[1] else grp_inf[2])

    calculate_chi_cross_tab(only_rel_f)
